<!-- 行列の分解（ぎょうれつのぶんかい，英: matrix decomposition, matrix factorization）とは，行列の行列の積への分解である -->
# 行列の分解とその他
Based on work by Andreas Noack

## 概要
 - 行列の分解
 - 特殊行列構造
 - 一般線形代数

線形系の準備から, 始める.

In [10]:
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 0.661002
 1.23433 
 1.88884 

## 行列の分解

<!-- 数学における行列のLU分解（エルユーぶんかい）とは、正方行列 A を下三角行列 L と上三角行列 U の積に分解すること。すなわち A = LU が成立するような L と U を求めることをいう。 -->
#### LU分解

Julia において, LU分解は以下のように示せる.

```julia
PA=LU
``` 

`P` は, 置換行列, `L` は, 下三角行列 単位対角, `U` は, `lufact` を使った上三角行列である. <!-- 疎行列LU分解 -->
<!-- 数学の一分野線型代数学における三角行列（さんかくぎょうれつ、英: triangular matrix）は特別な種類の正方行列である。正方行列が 下半三角または下三角であるとは主対角線より「上」の成分がすべて零となるときに言い、同様に上半三角または上三角とは主対角線より「下」の成分がすべて零となるときに言う。三角行列は上半または下半三角となる行列のことを言い、また上半かつ下半三角となる行列は対角行列と呼ぶ。
三角行列に関する行列方程式は解くことが容易であるから、それは数値解析において非常に重要である。LU分解アルゴリズムにより、正則行列が下半三角行列 L と上半三角行列 U との積 LU に書くことができるための必要十分条件は、その行列の首座小行列式 (leading principal minor) がすべて非零となることである。 -->

Juliaは, LU分解を計算することを可能にし,それを格納する複合分解タイプを定義する。

In [2]:
Alu = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}} with factors L and U:
[1.0 0.0 0.0; 0.111065 1.0 0.0; 0.633511 0.538573 1.0]
[0.969854 0.675868 0.504503; 0.0 0.419809 0.758999; 0.0 0.0 -0.527266]

In [3]:
typeof(Alu)

Base.LinAlg.LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by special indexing

In [4]:
Alu[:P]

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [5]:
Alu[:L]

3×3 Array{Float64,2}:
 1.0       0.0       0.0
 0.111065  1.0       0.0
 0.633511  0.538573  1.0

In [6]:
Alu[:U]

3×3 Array{Float64,2}:
 0.969854  0.675868   0.504503
 0.0       0.419809   0.758999
 0.0       0.0       -0.527266

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [8]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [ ]:
Aqr = qrfact(A)

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [ ]:
Aqr[:Q]

In [ ]:
Aqr[:R]

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [ ]:
Asym = A + A'
AsymEig = eigfact(Asym)

The values and the vectors can be extracted from the Eigen type by special indexing

In [ ]:
AsymEig[:values]

In [ ]:
AsymEig[:vectors]

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [ ]:
inv(AsymEig)*AsymEig

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [ ]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [ ]:
Asym = A + A'
issymmetric(Asym)

but sometimes floating point error might get in the way.

In [ ]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

In [ ]:
issymmetric(Asym_noisy)

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [ ]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [ ]:
@time eigvals(Asym);

In [ ]:
@time eigvals(Asym_noisy);

In [ ]:
@time eigvals(Asym_explicit);

In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` type to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [ ]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [ ]:
1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [ ]:
Arational = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

In [ ]:
x = fill(1, (3))
b = Arational*x

In [ ]:
Arational\b

In [ ]:
lufact(Ar)

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

#### 11.3 
Create a `LowerTriangular` matrix from `A`.